In [20]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import matplotlib
import os,sys,glob
from math import *
import xarray as xr
import pickle
import pytz
from datetime import datetime as dt
from datetime import timedelta, timezone
from scipy import optimize
import matplotlib.image as mpimg

In [2]:
# Load own modules
projectname = 'EUREC4A_organization'
workdir = os.getcwd()
rootdir = os.path.dirname(workdir)
while os.path.basename(rootdir) != projectname:
    rootdir = os.path.dirname(rootdir)
repodir = rootdir
moduledir = os.path.join(repodir,'functions')
resultdir = os.path.join(repodir,'results','idealized_calculations')
figdir = os.path.join(repodir,'figures','idealized_calculations')
inputdir = '/Users/bfildier/Dropbox/Data/EUREC4A/sondes_radiative_profiles/'
resultinputdir = os.path.join(repodir,'results','radiative_features')
radinputdir = os.path.join(repodir,'input')
scriptsubdir = 'observed_moist_intrusions'

os.makedirs(os.path.join(figdir,scriptsubdir),exist_ok=True)

In [7]:
# current environment
thismodule = sys.modules[__name__]

##-- Own modules
sys.path.insert(0,moduledir)
print("Own modules available:", [os.path.splitext(os.path.basename(x))[0]
                                 for x in glob.glob(os.path.join(moduledir,'*.py'))])

#- Parameters & constants
from thermoConstants import *
from thermoFunctions import *
from matrixoperators import *
from radiativefeatures import *
from radiativescaling import *

## Graphical parameters
plt.style.use(os.path.join(matplotlib.get_configdir(),'stylelib/presentation.mplstyle'))

Own modules available: ['conditionalstats', 'thermoFunctions', 'thermoConstants', 'radiativescaling', 'radiativefeatures', 'matrixoperators']


In [9]:
#-- day-by-day metadata

days =          '20200122','20200124','20200126','20200128','20200131','20200202','20200205','20200207','20200209','20200211','20200213'
name_pattern =  'Fish',    'Fish',    'Fish',    'Gravel',  'Fish',    'Flower',  'Gravel',  'Flower',    'Sugar',   'Sugar',   'Fish'
confidence_pattern = 'High','Medium', 'Medium',     'Low',     'Low',     'High',    'High',    'High',  'Medium',  'Medium',  'High'
col_pattern = {'':'silver',
               'Fish':'navy',
               'Gravel':'orange',
               'Sugar':'seagreen',
               'Flower':'firebrick'}

dim_t,dim_z = 0,1

# box GOES images
lat_box_goes = 10,16
lon_box_goes = -60,-52

# box of analysis
lat_box = 11,16
lon_box = -60,-52

# HALO circle coordinates
lon_center = -57.717
lat_center = 13.3
lon_pt_circle = -57.245
lat_pt_circle = 14.1903
r_circle = np.sqrt((lon_pt_circle - lon_center) ** 2 +
                   (lat_pt_circle - lat_center) ** 2)

# varids
ref_varid = 'PW'
cond_varids = 'QRAD','QRADSW','QRADLW','QV','UNORM','T','P'

In [21]:
###--- Load data ---###

# Profiles
radprf = xr.open_dataset(os.path.join(inputdir,'rad_profiles_CF.nc'))
# choose profiles for that day that start at bottom
data_all = radprf.where(radprf.z_min<=50,drop=True)

z = data_all.alt.values/1e3 # km
pres = np.nanmean(data_all.pressure.data,axis=dim_t)/100 # hPa

rad_features_all = {}
rad_scaling_all = {}
ref_dist_all = {}
cond_dist_all = {}

# initalize
for cond_varid in cond_varids:
     cond_dist_all[cond_varid] = {}

for day in days:

    #-- Radiative features
    features_filename = 'rad_features.pickle'
    print('loading %s'%features_filename)
    # load
    features_path = os.path.join(resultinputdir,day,features_filename)
    f = pickle.load(open(features_path,'rb'))
    # store
    rad_features_all[day] = f

    #-- Radiative scaling
    rad_scaling_filename = 'rad_scaling.pickle'
    print('loading %s'%rad_scaling_filename)
    rad_scaling_path = os.path.join(resultinputdir,day,rad_scaling_filename)
    rs = pickle.load(open(rad_scaling_path,'rb'))
    # store
    rad_scaling_all[day] = rs

    #-- Reference PW distribution
    ref_filename = 'dist_%s.pickle'%(ref_varid)
    print('load reference %s distribution'%ref_varid)
    ref_dist_path = os.path.join(resultinputdir,day,ref_filename)
    ref_dist = pickle.load(open(ref_dist_path,'rb'))
    # save in current environment
    ref_dist_all[day] = ref_dist

    #-- Conditional distributions
    for cond_varid in cond_varids:

        # load
        cond_filename = 'cdist_%s_on_%s.pickle'%(cond_varid,ref_varid)
        print('loading %s'%cond_filename)
        cond_dist_path = os.path.join(resultinputdir,day,cond_filename)
        cond_dist = pickle.load(open(cond_dist_path,'rb'))
        # save in current environment
        cond_dist_all[cond_varid][day] = cond_dist

# Bounds
ref_var_min = ref_dist_all['20200122'].bins[0]
ref_var_max = ref_dist_all['20200122'].bins[-1]

# load GOES images
date = dt(2020,1,26)
indir_goes_images = '/Users/bfildier/Data/satellite/GOES/images/%s'%date.strftime('%Y_%m_%d')
image_vis_files = glob.glob(os.path.join(indir_goes_images,'*C02*00.jpg'))
image_vis_files.sort()
# in the visible channel
images_vis = []
for i in range(len(image_vis_files)):
    images_vis.append(mpimg.imread(image_vis_files[i]))

# load Caroline's data for rad circulation
c_inputdir = os.path.join(repodir,'input','Caroline')
c_muller = scipy.io.loadmat(os.path.join(c_inputdir,'Qrad_pwbinnedvariables_ir90_t40_a50_nbins64.mat'))

# load moist intrusion data
# with piecewise linear fit and removed intrusions
rad_file_MI_20200213 = os.path.join(radinputdir,'rad_profiles_moist_intrusions_20200213.nc')
radprf_MI_20200213 = xr.open_dataset(rad_file_MI_20200213)

# with rectangular intrusions
rad_file_RMI_20200213 = os.path.join(radinputdir,'rad_profiles_rectangular_moist_intrusions.nc')
radprf_RMI_20200213 = xr.open_dataset(rad_file_RMI_20200213)



/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':


loading rad_features.pickle
loading rad_scaling.pickle
load reference PW distribution
loading cdist_QRAD_on_PW.pickle
loading cdist_QRADSW_on_PW.pickle
loading cdist_QRADLW_on_PW.pickle
loading cdist_QV_on_PW.pickle
loading cdist_UNORM_on_PW.pickle
loading cdist_T_on_PW.pickle
loading cdist_P_on_PW.pickle
loading rad_features.pickle
loading rad_scaling.pickle
load reference PW distribution
loading cdist_QRAD_on_PW.pickle
loading cdist_QRADSW_on_PW.pickle
loading cdist_QRADLW_on_PW.pickle
loading cdist_QV_on_PW.pickle
loading cdist_UNORM_on_PW.pickle
loading cdist_T_on_PW.pickle
loading cdist_P_on_PW.pickle
loading rad_features.pickle
loading rad_scaling.pickle
load reference PW distribution
loading cdist_QRAD_on_PW.pickle
loading cdist_QRADSW_on_PW.pickle
loading cdist_QRADLW_on_PW.pickle
loading cdist_QV_on_PW.pickle
loading cdist_UNORM_on_PW.pickle
loading cdist_T_on_PW.pickle
loading cdist_P_on_PW.pickle
loading rad_features.pickle
loading rad_scaling.pickle
load reference PW distri